# 3. Calculate variance of Stokes and anti-Stokes measurements

In [ ]:
import os

from dtscalibration import read_xml_dir

# from matplotlib import pyplot as plt

In [ ]:
try:
    wd = os.path.dirname(os.path.realpath(__file__))
except:
    wd = os.getcwd()

filepath = os.path.join(wd, '..', '..', 'tests', 'data', 'double_ended2')
timezone_netcdf = 'UTC',
timezone_ultima_xml = 'Europe/Amsterdam'
file_ext = '*.xml'

ds = read_xml_dir(filepath,
                  timezone_netcdf=timezone_netcdf,
                  timezone_ultima_xml=timezone_ultima_xml,
                  file_ext=file_ext)
sections = {
    'probe1Temperature': [slice(7.5, 17.), slice(70., 80.)],  # cold bath
    'probe2Temperature': [slice(24., 34.), slice(85., 95.)],  # warm bath
    }

In [ ]:
print(ds.variance_stokes.__doc__)

In [ ]:
I_var, residuals = ds.variance_stokes(st_label='ST', sections=sections, use_statsmodels=False)
print("The variance of the Stokes signal along the reference sections "
      "is approximately {}".format(I_var))

In [ ]:
# plt.hist(residuals, bins=50, density=True);

In [ ]:
# plt.plot(residuals)  # not precisely randoms

In [ ]:
import numpy as np

In [ ]:
np.tile([0., -1.], 4
       )